In [ ]:
import pandas as pd
import numpy as np

# Listing

In [ ]:
listing = pd.read_csv('raw_data/listings.csv.gz')

listing.info()

## Dimension: Amenities

In [ ]:
dim_amenities = listing[['id', 'amenities']].copy()

dim_amenities = dim_amenities.assign(amenities = dim_amenities['amenities'].str.split(',')).explode('amenities')
dim_amenities['amenities'] = dim_amenities['amenities'].str.replace(r'\[|\]|\"', '', regex = True).str.strip()

dim_amenities.drop_duplicates(inplace=True)

dim_amenities = dim_amenities[ ~dim_amenities['amenities'].isin(['', 'N/A'])]

dim_amenities['amenities'] = (
    dim_amenities['amenities']
    .astype(str)
    .str.encode('utf-8')
    .str.decode('unicode_escape')
)

dim_amenities.to_csv('clean_data/dim_amenities.csv', index=False)

Create a list of listing id for selected amenities.

In [ ]:
listing_with_heating = dim_amenities[ dim_amenities['amenities'].str.lower().str.contains('heating')]
listing_with_kitchen = dim_amenities[ (dim_amenities['amenities'].str.lower().str.contains('kitchen')) & (dim_amenities['amenities'] != 'Kitchenette') ]
listing_with_washer = dim_amenities[ dim_amenities['amenities'].str.lower().str.contains('washer')]
listing_with_wifi = dim_amenities[ dim_amenities['amenities'].str.lower().str.contains('wifi')]
listing_with_tv = dim_amenities[ dim_amenities['amenities'].str.lower().str.contains('tv')]

## Dimension: Listing

In [ ]:
dim_listing = listing[['id', 'host_id',
                       'listing_url', 'name', 'picture_url', 'description', 'instant_bookable', 
                        'price', 'property_type', 'room_type', 
                        'accommodates', 'beds', 'bedrooms', 'bathrooms', 'bathrooms_text',
                        'latitude', 'longitude', 'neighbourhood_cleansed', 
                        'number_of_reviews', 'review_scores_rating', 'review_scores_accuracy', 'review_scores_cleanliness', 'review_scores_checkin',
                        'review_scores_communication', 'review_scores_location', 'review_scores_value']].copy()

# Add new columns based on amenities
dim_listing['has_wifi'] = dim_listing['id'].isin(listing_with_wifi['id'])
dim_listing['has_tv'] = dim_listing['id'].isin(listing_with_tv['id'])
dim_listing['has_washer'] = dim_listing['id'].isin(listing_with_washer['id'])
dim_listing['has_heating_system'] = dim_listing['id'].isin(listing_with_heating['id'])
dim_listing['has_kitchen'] = dim_listing['id'].isin(listing_with_kitchen['id'])

str_column = dim_listing.select_dtypes('object').columns

for col in str_column:
    dim_listing.loc[:, col] = dim_listing[col].where(~dim_listing[col].isin(["", "N/A"]), None)

dim_listing['instant_bookable'] = dim_listing['instant_bookable'].map({'t': True, 'f': False})

dim_listing['price'] = dim_listing['price'].str.replace(r'\$|[,]', '', regex = True)
dim_listing['price'] = dim_listing['price'].where(dim_listing['price'].notna(), None)
dim_listing['price'] = dim_listing['price'].astype('Float64')


dim_listing.to_csv('clean_data/dim_listing.csv', index = False)

## Dimension: Host

In [ ]:
dim_host = listing[['host_id', 
                    'host_name',
                    'host_picture_url',
                    'host_identity_verified',
                    'host_is_superhost',
                    'host_neighbourhood',
                    'host_response_time',
                    'host_response_rate',
                    'host_since',
                    'host_about',
                    'host_url']].copy()

dim_host.drop_duplicates(inplace=True)

str_column = dim_host.select_dtypes('object').columns

for col in str_column:
    dim_host.loc[:, col] = dim_host[col].where(~dim_host[col].isin(["", "N/A"]), None)

dim_host['host_since'] = pd.to_datetime(dim_host['host_since'])
dim_host['host_response_rate'] = dim_host['host_response_rate'].str.replace('%', '')
dim_host['host_response_rate'] = dim_host['host_response_rate'].astype('Int64')

for col in ['host_identity_verified', 'host_is_superhost']:
    dim_host[col] = dim_host[col].map({'t': True, 'f': False})

dim_host['host_is_superhost'] = np.where(dim_host['host_is_superhost'].isna(), False, dim_host['host_is_superhost'])

dim_host.to_csv('clean_data/dim_host.csv', index = False)

# Calendar

In [ ]:
calendar = pd.read_csv('data/calendar.csv.gz')

calendar['available'] = calendar['available'].map({'f': False, 't':True})
calendar['date'] = pd.to_datetime(calendar['date'])

calendar.drop(['adjusted_price', 'price'], axis = 1, inplace=True)

calendar.to_csv('clean_data/fact_calendar.csv', index=False)